In [19]:
from paddlenlp import Taskflow
import paddle
import pandas as pd
import ast
from datetime import datetime

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


In [20]:
text = ''
# files_path = r'../../temp/甘肃省_学习考察.xlsx'
files_path = r'../../temp/全对测试集_学习考察.xlsx'
# paddle.device.set_device("gpu")
original_data = pd.read_excel(files_path)
data = original_data.copy()

In [21]:
def process_content(x):
    # 防止长度过长
    if len(x) > 1500:
        return x
    return ','.join(x)

data['content'] = data['content'].apply(ast.literal_eval)
data['content'] = data['content'].apply(process_content)

In [22]:
# 关系抽取
schema = {'考察人姓名': ['考察人职位', '考察人所属城市', '所考察的城市']}
ie = Taskflow('information_extraction', schema=schema, batch_size=8)

In [23]:
# schema = {'考察触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '访问触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '学习触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人']
#           }

# 批量处理数据
results = [ie(content) for content in data['content']]

In [ ]:
results

In [ ]:
# 将结果展开并格式化为 DataFrame 的行
formatted_results = []
for i, content in enumerate(data['content']):
    for item in results[i]:
        for person in item.get('考察人姓名', []):
            result_row = {
                'content': content,
                '人物': person.get('text', ''),
                '职位': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('考察人职位', [])]),
                '所属城市': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('考察人所属城市', [])]),
                '考察城市': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('所考察的城市', [])])
            }
            formatted_results.append(result_row)
            
# 用于存储去重后的结果
unique_results = {}

# 遍历每一条记录，按人物去重，并保留信息最多的那一项
for result in formatted_results:
    person = result['人物']
    
    # 如果该人物尚未在字典中，或当前记录信息更多，则更新字典
    if person not in unique_results or (
        len(result['职位']) + len(result['所属城市']) + len(result['考察城市']) >
        len(unique_results[person]['职位']) + len(unique_results[person]['所属城市']) + len(unique_results[person]['考察城市'])
    ):
        unique_results[person] = result

# 创建 DataFrame
results_df = pd.DataFrame(list(unique_results.values()))

current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# 保存为 CSV 文件
output_path = f'{current_time}_extracted_relations.csv'
results_df.to_csv(output_path, index=False, encoding='utf-8-sig')